In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install openai==0.28.0

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [2]:
import pandas as pd

# Replace the path with the location of your dataset in Google Drive
data_path = '/content/drive/MyDrive/NLP_fall_2024/russian-data-raw.csv'
df = pd.read_csv(data_path)

# Split the dataset into training (80%) and testing (20%)
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print("After split, train_df columns:", train_df.columns)

After split, train_df columns: Index(['id', 'text', 'label', 'notes'], dtype='object')


In [3]:
import os
from PIL import Image
import numpy as np

def load_image(image_id, img_folder_path):
    image_path = os.path.join(img_folder_path, f"{image_id}.png")
    if not os.path.exists(image_path):  # Try jpg/jpeg if png is not available
        for ext in ['jpg', 'jpeg']:
            image_path = os.path.join(img_folder_path, f"{image_id}.{ext}")
            if os.path.exists(image_path):
                break
    image = Image.open(image_path).convert('RGB').resize((224, 224))
    return np.array(image)

# Specify the folder containing your images
img_folder_path = '/content/drive/MyDrive/NLP_fall_2024/russian-images-archive'
print("After image", train_df.columns)

After image Index(['id', 'text', 'label', 'notes'], dtype='object')


In [4]:
import pandas as pd
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/NLP_fall_2024/russian-data-raw.csv")

print("Original DataFrame columns:", df.columns)
print("First few rows of data:\n", df.head())


# Extract the text column as a list
texts = df['text'].tolist()

# Initialize the tokenizer with a BPE model
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Train the tokenizer on your text data
trainer = trainers.BpeTrainer(vocab_size=30000, min_frequency=2)
tokenizer.train_from_iterator(texts, trainer)

# Save the tokenizer if needed
tokenizer.save("custom_tokenizer.json")


Original DataFrame columns: Index(['id', 'text', 'label', 'notes'], dtype='object')
First few rows of data:
    id                                               text  label  notes
0   1  Открыл рецепт простого борща? Поздравляю, ты о...      1    NaN
1   2  если я когда-нибудь напишу книгу, она будет на...      1    NaN
2   3  осенний суп. вкусно и просто за 5 минут! сохра...      1    NaN
3   4                 - как ты? - ниче, держусь, работаю      1    NaN
4   5  надписи на футболке: камень, бумага, ножницы, ...      1    NaN


In [5]:
import os
from PIL import Image
import numpy as np

print("Files in image folder:")
print(os.listdir(img_folder_path))

def load_image(image_id, img_folder_path):
    for ext in ['png', 'jpg', 'jpeg']:
        image_path = os.path.join(img_folder_path, f"{image_id}.{ext}")
        if os.path.exists(image_path):
            image = Image.open(image_path).convert('RGB').resize((224, 224))
            print(f"Loaded image: {image_path}")  # Print confirmation
            return np.array(image)
    print(f"Image not found for ID: {image_id}")  # Print if no image found
    return None

for idx in range(5):  # Adjust range as needed
    image = load_image(train_df.iloc[idx]['id'], img_folder_path)
    if image is None:
        print(f"Image with ID {train_df.iloc[idx]['id']} not loaded.")
    else:
        print(f"Image with ID {train_df.iloc[idx]['id']} loaded successfully.")

Files in image folder:
['35.jpg', '36.jpg', '37.jpg', '67.png', '38.jpg', '68.png', '69.png', '70.png', '72.png', '71.png', '73.png', '74.png', '75.png', '76.png', '39.png', '77.png', '40.png', '78.png', '41.png', '42.png', '79.png', '43.png', '80.png', '44.png', '81.png', '45.png', '46.png', '47.png', '49.png', '48.png', '50.png', '52.png', '51.png', '53.png', '54.png', '82.png', '83.png', '57.png', '55.png', '56.png', '58.png', '60.png', '84.png', '61.png', '62.png', '63.png', '59.png', '21.jpg', '22.jpg', '23.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '24.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '1.jpg', '2.jpg', '3.jpg', '5.jpg', '10.jpg', '11.jpg', '4.jpg', '13.jpg', '12.jpg', '8.jpg', '6.jpg', '7.jpg', '14.jpg', '15.jpg', '16.jpg', '18.jpg', '19.jpg', '20.jpg', '17.jpg', '34.jpg', '89.png', '90.png', '91.jpg', '92.png', '93.png', '94.png', '95.png', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '108.jpg', '109.jpg', '110.jpg', '111

In [ ]:
import openai

openai.api_key = "hidden"

# def get_embedding(text):
#     response = openai.Embedding.create(input=text, model="gpt-3.5-turbo")
#     return response['data'][0]['embedding']


def get_representation(text, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "Generate a concise representation for embeddings."},
                {"role": "user", "content": text}
            ]
        )
        # Use response to get text for representation
        representation = response.choices[0].message['content']
        # You might want to tokenize this representation into word vectors or simple sentence embedding
        return representation
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage
df['representation'] = df['text'].apply(get_representation)

print(df[['text', 'representation']].head())
print(train_df.columns)


                                                text  \
0  Открыл рецепт простого борща? Поздравляю, ты о...   
1  если я когда-нибудь напишу книгу, она будет на...   
2  осенний суп. вкусно и просто за 5 минут! сохра...   
3                 - как ты? - ниче, держусь, работаю   
4  надписи на футболке: камень, бумага, ножницы, ...   

                                      representation  
0  Пользователь открыл рецепт борща. На конференц...  
1  Книга: "Бегущая по граблям" #дневникидуры #юмо...  
2  Рецепт: осенний суп за 5 минут. #юмор #черныйю...  
3                                   Хорошо, работаю.  
4  Футболка: камень, бумага, ножницы, циркулярка ...  
Index(['id', 'text', 'label', 'notes'], dtype='object')


In [7]:
# Step 1: Generate representation on the full df
df['representation'] = df['text'].apply(get_representation)

# Step 2: Split df into train and test sets with the new representation column
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Final Checks
print("train_df columns after generating representations:", train_df.columns)
print("Sample rows from train_df with 'representation' column:")
print(train_df[['text', 'representation']].head())

# Check for any missing values in 'representation'
missing_representations = df['representation'].isnull().sum()
print(f"Number of missing representations: {missing_representations}")


train_df columns after generating representations: Index(['id', 'text', 'label', 'notes', 'representation'], dtype='object')
Sample rows from train_df with 'representation' column:
                                                  text  \
137                              Наш декоративный очаг   
5    Санёк, что ты носишься? Премию не дадут, подза...   
66   КАК КАЖДУЮ НЕДЕЛЮ ОЩУЩАЕТСЯ ПОДВИГ В ВИДЕ НАХО...   
138  Малость приболел Решил поесть Спасибо за внимание   
104  🐿 В парке Металлургов позирует всем вот такая ...   

                                        representation  
137                               decorative fireplace  
5    Шутник предполагает, что неценные похвалы возд...  
66   Weekly household cleaning feels like a challen...  
138  Feeling slightly sick, decided to eat. Thank y...  
104  Красавица в парке Металлургов, Липецк 🐿 #липец...  
Number of missing representations: 0


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader

class SarcasmDataset(Dataset):
    def __init__(self, df, img_folder_path):
        self.df = df
        self.img_folder_path = img_folder_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Use the tokenized representation
        embedding = torch.tensor(row['tokenized_representation'], dtype=torch.long)

        # Load image and ensure it has the correct shape
        image = load_image(row['id'], self.img_folder_path)
        if image is None:
            print(f"Image not found for ID {row['id']}")
            return None
        image = torch.from_numpy(image).float().permute(2, 0, 1).clone().detach() / 255.0  # Shape: [3, 224, 224]

        # Load label
        label = torch.tensor(row['label'], dtype=torch.long)

        return embedding, image, label


In [9]:
# Check columns in train_df and test_df before creating the dataset
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

# Verify the first few rows to ensure that 'representation' column has values
print("Sample rows from train_df:")
print(train_df[['id', 'text', 'label', 'representation']].head())

print("Sample rows from test_df:")
print(test_df[['id', 'text', 'label', 'representation']].head())

# Now create the SarcasmDataset instance after confirming the columns
train_dataset = SarcasmDataset(train_df, img_folder_path)
test_dataset = SarcasmDataset(test_df, img_folder_path)

# Initialize DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Columns in train_df: Index(['id', 'text', 'label', 'notes', 'representation'], dtype='object')
Columns in test_df: Index(['id', 'text', 'label', 'notes', 'representation'], dtype='object')
Sample rows from train_df:
      id                                               text  label  \
137  140                              Наш декоративный очаг      0   
5      6  Санёк, что ты носишься? Премию не дадут, подза...      1   
66    69  КАК КАЖДУЮ НЕДЕЛЮ ОЩУЩАЕТСЯ ПОДВИГ В ВИДЕ НАХО...      1   
138  141  Малость приболел Решил поесть Спасибо за внимание      0   
104  107  🐿 В парке Металлургов позирует всем вот такая ...      0   

                                        representation  
137                               decorative fireplace  
5    Шутник предполагает, что неценные похвалы возд...  
66   Weekly household cleaning feels like a challen...  
138  Feeling slightly sick, decided to eat. Thank y...  
104  Красавица в парке Металлургов, Липецк 🐿 #липец...  
Sample rows from test

In [10]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Define a function to tokenize each representation
def tokenize_representation(text):
    tokens = tokenizer(text, padding='max_length', max_length=128, truncation=True, return_tensors="pt")
    return tokens['input_ids'].squeeze(0).tolist()  # Convert to list for each row

# Apply tokenization to each row in the 'representation' column
df['tokenized_representation'] = df['representation'].apply(tokenize_representation)

# Check the result
print("Sample rows with tokenized representations:")
print(df[['representation', 'tokenized_representation']].head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Sample rows with tokenized representations:
                                      representation  \
0  Рецепт борща + конференция по ядерной физике и...   
1  "Бегущая по граблям" - книга о юморе, сарказме...   
2  Рецепт: Осенний суп за 5 минут. #юмор #черныйю...   
3              Состояние: нормальное, занят работой.   
4  Футболка: камень, бумага, ножницы, циркулярка....   

                            tokenized_representation  
0  [101, 1195, 15290, 29751, 15290, 29746, 22919,...  
1  [101, 1000, 1181, 15290, 29741, 29748, 29754, ...  
2  [101, 1195, 15290, 29751, 15290, 29746, 22919,...  
3  [101, 1196, 14150, 29747, 22919, 14150, 17432,...  
4  [101, 1199, 29748, 22919, 29740, 14150, 29436,...  


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
import os
from PIL import Image
import torch

def load_image(image_id, img_folder_path):
    # Check for common image file formats
    for ext in ['png', 'jpg', 'jpeg']:
        image_path = os.path.join(img_folder_path, f"{image_id}.{ext}")
        if os.path.exists(image_path):
            # Load and resize the image to (224, 224)
            image = Image.open(image_path).convert('RGB').resize((224, 224))
            return torch.tensor(np.array(image), dtype=torch.float32).permute(2, 0, 1) / 255.0
    return None  # Return None if the image isn't found

In [12]:
print(train_df.columns)


Index(['id', 'text', 'label', 'notes', 'representation'], dtype='object')


In [13]:
from torch.utils.data import Dataset, DataLoader

class SarcasmDataset(Dataset):
    def __init__(self, df, img_folder_path):
        self.df = df
        self.img_folder_path = img_folder_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Check if the image exists
        image = load_image(row['id'], self.img_folder_path)
        if image is None:
            # Skip this index by returning the next available sample
            return self.__getitem__((idx + 1) % len(self))

        # Get tokenized text and label
        tokenized_text = torch.tensor(row['tokenized_text'], dtype=torch.long)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        label = torch.tensor(row['label'], dtype=torch.long)

        return tokenized_text, image, label



In [14]:
print(df.columns)
print(df[['text', 'representation']].head())

Index(['id', 'text', 'label', 'notes', 'representation',
       'tokenized_representation'],
      dtype='object')
                                                text  \
0  Открыл рецепт простого борща? Поздравляю, ты о...   
1  если я когда-нибудь напишу книгу, она будет на...   
2  осенний суп. вкусно и просто за 5 минут! сохра...   
3                 - как ты? - ниче, держусь, работаю   
4  надписи на футболке: камень, бумага, ножницы, ...   

                                      representation  
0  Рецепт борща + конференция по ядерной физике и...  
1  "Бегущая по граблям" - книга о юморе, сарказме...  
2  Рецепт: Осенний суп за 5 минут. #юмор #черныйю...  
3              Состояние: нормальное, занят работой.  
4  Футболка: камень, бумага, ножницы, циркулярка....  


In [16]:
import os

# Define a function to check if an image exists
def image_exists(image_id, folder_path):
    return any(os.path.exists(os.path.join(folder_path, f"{image_id}.{ext}")) for ext in ['png', 'jpg', 'jpeg'])

train_df = train_df[train_df['id'].apply(lambda x: image_exists(x, img_folder_path))].reset_index(drop=True)
test_df = test_df[test_df['id'].apply(lambda x: image_exists(x, img_folder_path))]

# Now recreate the Dataset and DataLoader
train_dataset = SarcasmDataset(train_df, img_folder_path)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

print("Filtered train_df columns:", train_df.columns)
print("Filtered train_df size:", len(train_df))

Filtered train_df columns: Index(['id', 'text', 'label', 'notes', 'representation'], dtype='object')
Filtered train_df size: 153


In [17]:
from tokenizers import Tokenizer

# Load the custom tokenizer
tokenizer = Tokenizer.from_file("custom_tokenizer.json")

# Tokenize text data in train and test sets
def tokenize_text(text):
    tokens = tokenizer.encode(text).ids  # Encode text as a list of token IDs
    return tokens

# Apply tokenization and pad sequences to a fixed length (e.g., 128 tokens)
max_length = 128

train_df['tokenized_text'] = train_df['text'].apply(lambda x: tokenize_text(x)[:max_length] + [0]*(max_length - len(tokenize_text(x))))
test_df['tokenized_text'] = test_df['text'].apply(lambda x: tokenize_text(x)[:max_length] + [0]*(max_length - len(tokenize_text(x))))


In [18]:
import torch
from torch.utils.data import Dataset

class SarcasmDataset(Dataset):
    def __init__(self, df, img_folder_path):
        self.df = df
        self.img_folder_path = img_folder_path

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Use 'tokenized_text' instead of 'embedding'
        tokenized_text = torch.tensor(row['tokenized_text'], dtype=torch.long)

        # Load the image based on ID
        image = load_image(row['id'], self.img_folder_path)
        if image is None:
            raise FileNotFoundError(f"Image with ID {row['id']} not found in {self.img_folder_path}")
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0  # Normalize to [0, 1]

        # Get the label
        label = torch.tensor(row['label'], dtype=torch.long)

        return tokenized_text, image, label


In [19]:
import torch.nn as nn

class MultimodalModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super(MultimodalModel, self).__init__()

        # Text embedding and linear layers
        self.text_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.fc_text = nn.Linear(embedding_dim * max_length, 128)

        # Image feature extraction (simple CNN)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )

        # Combine both modalities
        self.fc_combined = nn.Linear(128 + 32 * 55 * 55, num_classes)  # Adjust if CNN output size changes

    def forward(self, tokenized_text, image):
        # Process text input
        embedded_text = self.text_embedding(tokenized_text).view(-1, embedding_dim * max_length)
        text_out = torch.relu(self.fc_text(embedded_text))

        # Process image input
        image_out = self.cnn(image)

        # Concatenate text and image outputs
        combined = torch.cat((text_out, image_out), dim=1)
        return self.fc_combined(combined)

# Define model parameters and instantiate the model
vocab_size = len(tokenizer.get_vocab())
embedding_dim = 64  # Adjust as needed
num_classes = 2
model = MultimodalModel(vocab_size, embedding_dim, num_classes)


In [20]:
# Check that the column exists and inspect the first few rows
print(train_df.columns)
print(train_df[['text', 'tokenized_text']].head())

Index(['id', 'text', 'label', 'notes', 'representation', 'tokenized_text'], dtype='object')
                                                text  \
0                              Наш декоративный очаг   
1  Санёк, что ты носишься? Премию не дадут, подза...   
2  КАК КАЖДУЮ НЕДЕЛЮ ОЩУЩАЕТСЯ ПОДВИГ В ВИДЕ НАХО...   
3  Малость приболел Решил поесть Спасибо за внимание   
4  🐿 В парке Металлургов позирует всем вот такая ...   

                                      tokenized_text  
0  [592, 206, 198, 991, 94, 287, 106, 241, 95, 0,...  
1  [524, 541, 102, 7, 229, 235, 1001, 271, 213, 2...  
2  [1226, 446, 67, 65, 80, 90, 521, 445, 72, 90, ...  
3  [630, 212, 256, 252, 574, 103, 641, 314, 103, ...  
4  [150, 63, 1223, 859, 1316, 1014, 1117, 597, 49...  


In [21]:
from sklearn.model_selection import train_test_split

# Randomly split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# Confirm the randomized split by inspecting a few rows
print("Training set sample:")
print(train_df.head())
print("\nTesting set sample:")
print(test_df.head())

Training set sample:
      id                                               text  label  notes  \
137  140                              Наш декоративный очаг      0    NaN   
5      6  Санёк, что ты носишься? Премию не дадут, подза...      1    NaN   
66    69  КАК КАЖДУЮ НЕДЕЛЮ ОЩУЩАЕТСЯ ПОДВИГ В ВИДЕ НАХО...      1    NaN   
138  141  Малость приболел Решил поесть Спасибо за внимание      0    NaN   
104  107  🐿 В парке Металлургов позирует всем вот такая ...      0    NaN   

                                        representation  \
137                               decorative fireplace   
5    Шутник предполагает, что неценные похвалы возд...   
66   Weekly household cleaning feels like a challen...   
138  Feeling slightly sick, decided to eat. Thank y...   
104  Красавица в парке Металлургов, Липецк 🐿 #липец...   

                              tokenized_representation  
137  [101, 11584, 13788, 102, 0, 0, 0, 0, 0, 0, 0, ...  
5    [101, 1203, 29748, 22919, 18947, 10325, 23925,..

In [37]:
# Pre-filtering the DataFrame to exclude rows with missing images
def filter_missing_images(df, img_folder_path):
    valid_indices = []
    for idx, row in df.iterrows():
        # Check if the image exists
        found_image = False
        for ext in ['png', 'jpg', 'jpeg']:
            image_path = os.path.join(img_folder_path, f"{row['id']}.{ext}")
            if os.path.exists(image_path):
                found_image = True
                break
        if found_image:
            valid_indices.append(idx)
        else:
            print(f"Image with ID {row['id']} not found. Excluding from dataset.")

    return df.loc[valid_indices].reset_index(drop=True)

# Filter both train and test DataFrames
train_df = filter_missing_images(train_df, img_folder_path)
test_df = filter_missing_images(test_df, img_folder_path)

# Now, create the dataset and DataLoader as before
train_dataset = SarcasmDataset(train_df, img_folder_path, tokenizer)
test_dataset = SarcasmDataset(test_df, img_folder_path, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


Image with ID 142 not found. Excluding from dataset.
Image with ID 147 not found. Excluding from dataset.
Image with ID 155 not found. Excluding from dataset.
Image with ID 9 not found. Excluding from dataset.
Image with ID 152 not found. Excluding from dataset.


In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import numpy as np
from PIL import Image
import os

# Function to load an image
def load_image(image_id, img_folder_path):
    for ext in ['png', 'jpg', 'jpeg']:
        image_path = os.path.join(img_folder_path, f"{image_id}.{ext}")
        if os.path.exists(image_path):
            image = Image.open(image_path).convert('RGB').resize((224, 224))
            return np.array(image)
    return None

# Dataset Class
class SarcasmDataset(Dataset):
    def __init__(self, df, img_folder_path, tokenizer):
        self.df = df
        self.img_folder_path = img_folder_path
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Tokenize text representation
        tokens = self.tokenizer(row['representation'], padding='max_length', max_length=128, truncation=True, return_tensors="pt")
        embedding = tokens['input_ids'].squeeze(0).float()  # Convert to float here

        # Load image and make sure it has the correct shape
        image = load_image(row['id'], self.img_folder_path)
        if image is None:
            raise ValueError(f"Image with ID {row['id']} not found.")
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0  # Convert to tensor and normalize

        # Load label
        label = torch.tensor(row['label'], dtype=torch.long)

        return embedding, image, label
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Initialize dataset and dataloaders
train_dataset = SarcasmDataset(train_df, img_folder_path, tokenizer)
test_dataset = SarcasmDataset(test_df, img_folder_path, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

# Define a simple model for testing purposes
class MultimodalModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MultimodalModel, self).__init__()

        # Text representation layers
        self.fc_text = nn.Linear(input_size, 128)

        # Image feature extraction (using simple CNN layers for illustration)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten()
        )

        # Calculate the flattened output size after CNN layers
        self.fc_combined = nn.Linear(128 + 32 * 55 * 55, num_classes)  # Adjust sizes to match CNN output

    def forward(self, representation, image):
        text_out = torch.relu(self.fc_text(representation))
        image_out = self.cnn(image)

        combined = torch.cat((text_out, image_out), dim=1)
        return self.fc_combined(combined)

# Initialize model, loss function, and optimizer
model = MultimodalModel(input_size=128, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 100  # Adjust as needed
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (tokenized_text, image, label) in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(tokenized_text, image)

        # Compute loss
        loss = criterion(outputs, label)
        running_loss += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {running_loss / len(train_loader):.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Epoch [1/100], Batch [1], Loss: 57.5050
Epoch [1/100], Batch [11], Loss: 0.0000
Epoch [1/100], Batch [21], Loss: 81.5490
Epoch [1/100], Batch [31], Loss: 19.4887
Epoch [1/100], Batch [41], Loss: 104.9653
Epoch [1/100], Batch [51], Loss: 243.6769
Epoch [1/100], Batch [61], Loss: 267.3404
Epoch [1/100], Batch [71], Loss: 101.4124
Epoch [1/100], Average Loss: 138.4742
Epoch [2/100], Batch [1], Loss: 332.1824
Epoch [2/100], Batch [11], Loss: 0.0000
Epoch [2/100], Batch [21], Loss: 145.0236
Epoch [2/100], Batch [31], Loss: 90.0998
Epoch [2/100], Batch [41], Loss: 113.8135
Epoch [2/100], Batch [51], Loss: 46.0518
Epoch [2/100], Batch [61], Loss: 0.0000
Epoch [2/100], Batch [71], Loss: 25.5493
Epoch [2/100], Average Loss: 43.3930
Epoch [3/100], Batch [1], Loss: 30.2890
Epoch [3/100], Batch [11], Loss: 159.5012
Epoch [3/100], Batch [21], Loss: 3.7096
Epoch [3/100], Batch [31], Loss: 0.0000
Epoch [3/100], Batch [41], Loss: 0.0000
Epoch [3/100], Batch [51], Loss: 0.0000
Epoch [3/100], Batch [61]

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Switch model to evaluation mode
model.eval()

# Track predictions and labels
all_preds = []
all_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for tokenized_text, image, label in test_loader:
        # Forward pass to get predictions
        outputs = model(tokenized_text, image)

        # Get the predicted class (use softmax for multi-class)
        _, predicted = torch.max(outputs, 1)

        # Append predictions and true labels for metrics calculation
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.6000
Precision: 0.6274
Recall: 0.6000
F1 Score: 0.5877
